In [1]:
print("ujjwal")

ujjwal


In [2]:
from langchain import PromptTemplate


In [3]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [7]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf", loader_cls=PyMuPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extraced_date = load_pdf("data/")

Create text chunks 

In [9]:
def text_splitter(extraced_date):
    text_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_split.split_documents(extraced_date)
    return text_chunks

In [10]:
text_chunks = text_splitter(extraced_date)
print("lenght of chunks", len(text_chunks))

lenght of chunks 23


In [11]:
def load_geneni_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY"))
    return embeddings

In [12]:
embeddings = load_geneni_embeddings()

In [13]:
embeddings 

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022BC7F31A50>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [14]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key= PINECONE_API_KEY)
index = pc.Index("bookchat")
vector_store= PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name="bookchat")


In [16]:
docsearch = PineconeVectorStore.from_existing_index(index_name="bookchat",embedding=embeddings)

query = "what social engineering"

docs = docsearch.similarity_search(query, k=3)
print("Result", docs)

Result [Document(id='bd629292-ce74-4567-841b-319f629ccd63', metadata={}, page_content='- **Social Engineering**: AI enhancing targeted scams through believable chatbots and voice \nassistants, potentially compromising software security measures. \n \n## Project Management AI-Driven Workflows \n \nAI-driven workflows are revolutionizing project management in software engineering, enhancing \nefficiency and accuracy. Key tools include: \n \n1. **Asana AI**: Automates tasks with AI-powered productivity recommendations and smart workflows, \nstreamlining software project management.'), Document(id='0e30a83b-84e4-47c0-96cd-fc1db3a524ba', metadata={}, page_content='- **Social Engineering**: AI enhancing targeted scams through believable chatbots and voice \nassistants, potentially compromising software security measures. \n \n## Project Management AI-Driven Workflows \n \nAI-driven workflows are revolutionizing project management in software engineering, enhancing \nefficiency and accuracy. 

In [17]:
PromptTemplates = """
 Use the following pieces of the information to answer the user's question.
 If you don't know the answer,just say you don't know, dont try to make up an answer.

 context: {context}
 Question: {question}

 Only retrun the helpful answer below and nothing else.
 Helpful Answer:
 """

In [18]:
from langchain.prompts import PromptTemplate

In [19]:
context_prompt=PromptTemplate(
    input_variables=["context", "question"],
    template=PromptTemplates,
)
chain_type_kwargs = {"prompt": context_prompt}

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

The below method has been deprecated so we can do like this also 

In [21]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
from langchain_core.runnables import RunnableMap
from langchain.chains import RetrievalQA
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

qa_chain = load_qa_with_sources_chain(llm, chain_type="stuff", **chain_type_kwargs).invoke(x["question"])
retrieval_qa_chain = RunnableMap({
    "input_documents": lambda x: docsearch.as_retriever(search_kwargs={"k": 2}).invoke(x["question"]),
    "question": lambda x: x["question"]
}) | qa_chain

while True:
    user_input = input("enter prompt:")
    result = retrieval_qa_chain.invoke(query)
    print("Result:", result['result])

In [ ]:
while True:
    user_input = input("Input Prompt: ")
    result = qa({"query": user_input})
    print("Result: ", result['result'])
    

C:\Users\HP 15\AppData\Local\Temp\ipykernel_16664\2197071624.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": user_input})


Result:  capabilities rather than viewing it as a replacement for human expertise and creativity.
Result:  I don't know.
